## 목표

- 네이버 영화리뷰 데이터셋을 이용해 긍정/부정 분류기를 만들어보자(0이면 부정 1이면 긍정)
- TF-IDF (단어의 빈도와 문서내의 빈도를 같이 고려) 방법을 적용해보자
- Konlpy(한국어 형태소 분석기)를 설치하고 활용해보자
- 단어별 긍정/부정 정보를 확인해보자

In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 92kB 7.5MB/s 
     |████████████████████████████████| 460kB 44.0MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


## 네이버 영화리뷰 데이터 로딩

In [14]:
import pandas as pd
# set_option : 데이터프레임의 환경을 설정해주는 명령
# display.max_colwidth : 최대 열 너비를 설정 (None으로 설정시 무제한)
# 영화 리뷰는 길어서 모두 볼수 있게
pd.set_option('display.max_colwidth', None)

In [15]:
# 데이터 프레임 화
df_train = pd.read_csv('/content/ratings_train.txt', delimiter = '\t')
df_test = pd.read_csv('/content/ratings_test.txt', delimiter = '\t')
# delimiter : 파일에서 데이터 사이의 간격을 구분해주는 구분자
# 기본값: ,

In [16]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [17]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [18]:
# dropna : 결측치 (null값)를 하나라도 포함하는 행을 제거
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

In [54]:
df_train.head(5)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다,1


In [19]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [20]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


- train,test 문제, 정답 으로 분류해보자

In [21]:
text_train = df_train['document']
y_train = df_train['label']
text_test = df_test['document']
y_test = df_test['label']

print(text_train.shape)
print(y_train.shape)
print(text_test.shape)
print(y_test.shape)

(149995,)
(149995,)
(49997,)
(49997,)


## BoW 기법 (1. 카운트벡터라이저, 2. TF-IDF)
1. 카운트벡터라이저 : 단순히 단어의 빈도수만 파악
2. TF-IDF : 단어의 빈도수 + 단어가 등장하는 문서의 개수 모두 파악

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
tmp_tf_idf = TfidfVectorizer()

In [24]:
# fit : bow(단어가방)으로 만들어주기
tmp_tf_idf.fit(text_train[:3]) # 3개의 행 까지만 단어가방으로 만들어보자

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [25]:
# vocabulary_ : 토큰화된 단어와 매핑된 인덱스를 확인
tmp_tf_idf.vocabulary_

{'가볍지': 0,
 '너무재밓었다그래서보는것을추천한다': 1,
 '더빙': 2,
 '목소리': 3,
 '않구나': 4,
 '오버연기조차': 5,
 '진짜': 6,
 '짜증나네요': 7,
 '초딩영화줄': 8,
 '포스터보고': 9}

In [26]:
# vocabulary_ : 토큰화된 단어만 보기
tmp_tf_idf.get_feature_names()

['가볍지',
 '너무재밓었다그래서보는것을추천한다',
 '더빙',
 '목소리',
 '않구나',
 '오버연기조차',
 '진짜',
 '짜증나네요',
 '초딩영화줄',
 '포스터보고']

### konlpy - kkma 명령 사용해보기
 - konlpy : 한글을 형태소마다 잘 토큰화 시킬 수 있는 도구들의 집합
 - kkma : konlpy 중 가장 많이 사용하는 도구

In [27]:
from konlpy.tag import Kkma

In [28]:
kkma = Kkma()

In [29]:
kkma.nouns(text_train[0]) # 명사만 토큰화 시키기

['더빙', '목소리']

In [30]:
# 실제 문장 넣어보자!
tmp = '오늘도 강준이는 성우쌤에게 그윽한 눈길을 보낸다'
kkma.nouns(tmp)

['오늘', '강', '이', '성우', '게', '눈길']

In [34]:
# 입력된 문장에 대해서 명사만 추출하는 함수
def myTokenizer(text) :
  return  kkma.nouns(text)

In [35]:
tmp_tf_idf = TfidfVectorizer(tokenizer=myTokenizer)
tmp_tf_idf.fit(text_train[:3])
tmp_tf_idf.vocabulary_

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


{'더빙': 0,
 '딩': 1,
 '목소리': 2,
 '밓': 3,
 '보고': 4,
 '연기': 5,
 '영화': 6,
 '오버': 7,
 '오버연기': 8,
 '재': 9,
 '재밓': 10,
 '줄': 11,
 '초': 12,
 '초딩영화줄': 13,
 '추천': 14,
 '포스터': 15,
 '포스터보고': 16,
 '흠': 17}

pos tagging 활용 (품사 매핑)
- 문장에서 각각의 단어를 해당하는 품사로 매핑하는 작업
- 한글 문장에서 단어들의 품사를 알아보자

In [36]:
data = '먹는다 먹다 먹었다 이쁘다 아름답다 사진 모자'
kkma.morphs(data) # 형태소로 분리

['먹', '는', '다', '먹', '다', '먹', '었', '다', '이쁘', '다', '아름답', '다', '사진', '모자']

In [37]:
kkma.pos(data) # 품사를 표시해주는 명령

[('먹', 'VV'),
 ('는', 'EPT'),
 ('다', 'ECS'),
 ('먹', 'VV'),
 ('다', 'ECS'),
 ('먹', 'VV'),
 ('었', 'EPT'),
 ('다', 'EFN'),
 ('이쁘', 'VA'),
 ('다', 'ECS'),
 ('아름답', 'VA'),
 ('다', 'ECS'),
 ('사진', 'NNG'),
 ('모자', 'NNG')]

In [38]:
kkma.tagset

{'EC': '연결 어미',
 'ECD': '의존적 연결 어미',
 'ECE': '대등 연결 어미',
 'ECS': '보조적 연결 어미',
 'EF': '종결 어미',
 'EFA': '청유형 종결 어미',
 'EFI': '감탄형 종결 어미',
 'EFN': '평서형 종결 어미',
 'EFO': '명령형 종결 어미',
 'EFQ': '의문형 종결 어미',
 'EFR': '존칭형 종결 어미',
 'EP': '선어말 어미',
 'EPH': '존칭 선어말 어미',
 'EPP': '공손 선어말 어미',
 'EPT': '시제 선어말 어미',
 'ET': '전성 어미',
 'ETD': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JK': '조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKI': '호격 조사',
 'JKM': '부사격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JX': '보조사',
 'MA': '부사',
 'MAC': '접속 부사',
 'MAG': '일반 부사',
 'MD': '관형사',
 'MDN': '수 관형사',
 'MDT': '일반 관형사',
 'NN': '명사',
 'NNB': '일반 의존 명사',
 'NNG': '보통명사',
 'NNM': '단위 의존 명사',
 'NNP': '고유명사',
 'NP': '대명사',
 'NR': '수사',
 'OH': '한자',
 'OL': '외국어',
 'ON': '숫자',
 'SE': '줄임표',
 'SF': '마침표, 물음표, 느낌표',
 'SO': '붙임표(물결,숨김,빠짐)',
 'SP': '쉼표,가운뎃점,콜론,빗금',
 'SS': '따옴표,괄호표,줄표',
 'SW': '기타기호 (논리수학기호,화폐기호)',
 'UN': '명사추정범주',
 'VA': '형용사',
 'VC': '지정사',
 'VCN': "부정 지정사, 형용사 '아니다'",
 'VC

In [39]:
d = pd.DataFrame(kkma.pos(data), columns=['morph','tag'])

In [40]:
d

,morph,tag
0,먹,VV
1,는,EPT
2,다,ECS
3,먹,VV
4,다,ECS
5,먹,VV
6,었,EPT
7,다,EFN
8,이쁘,VA
9,다,ECS


In [41]:
d.set_index('tag', inplace=True) # 기존의 인덱스를 제거하고 'tag' 컬럼을 인덱스로 설정
d.loc[['VV','VA','NNG']] # VV: 동사. VA: 형용사. NNG 명사 행만 출력하기

,morph
tag,
VV,먹
VV,먹
VV,먹
VA,이쁘
VA,아름답
NNG,사진
NNG,모자


In [44]:
def myTokenizer2(text) :
  d = pd.DataFrame(kkma.pos(data), columns=['morph','tag'])
  d.set_index('tag', inplace = True)
  if ('VV' in d.index) | ('VA' in d.index) | ('NNG' in d.index) :
    labels = ['VV', 'VA', 'NNG']
    # 파이썬 형식 사용
    return d.loc[d.index.intersection(labels)].dropna()['morph'].values
  else :
    return []

In [45]:
tmp_tf_idf = TfidfVectorizer(tokenizer=myTokenizer2)
tmp_tf_idf.fit(text_train[:3])
tmp_tf_idf.vocabulary_

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


{'먹': 0, '모자': 1, '사진': 2, '아름답': 3, '이쁘': 4}

In [48]:
final_tf_idf = TfidfVectorizer(tokenizer=myTokenizer2)
final_tf_idf.fit(text_train[:5000])

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function myTokenizer2 at 0x7fd1baeadd90>,
                use_idf=True, vocabulary=None)

In [53]:
len(final_tf_idf.vocabulary_)

5

In [49]:
# 토큰화 된거 수치화
X_train = tmp_tf_idf.transform(text_train[:1000])
X_test = tmp_tf_idf.transform(text_test[:1000])

In [50]:
from sklearn.linear_model import LogisticRegression

In [51]:
logi = LogisticRegression()
logi.fit(X_train, y_train[:1000])
logi.score(X_test, y_test[:1000])

0.492